In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
import matplotlib.gridspec as gridspec

from astropy.io import fits
from astropy.wcs import WCS
from kcwitools.io import open_kcwi_cube
import kcwitools.extract_weighted_spectrum as ews
from kcwitools.utils import build_wave
# from musetools.utils import transform_pixels
def transform_pixels(x1, y1, wcs1, wcs2):
    r, d = wcs1.pixel_to_world_values(x1, y1)
    x2, y2 = wcs2.world_to_pixel_values(r, d)
    return x2, y2
def find_fits(basedir, endswith):
    return (os.path.join(root, file)
        for root, dirs, files in os.walk(basedir)
            for file in files if root == basedir and file.lower().endswith(endswith))
def find_files_ending_with(directory, endswith):
    # List all files in the specified directory
    files = os.listdir(directory)
    # Filter the list to only include files that end with the specified suffix
    matching_files = [file for file in files if file.endswith(endswith)]
    return matching_files
def find_fits(basedir, endswith):
    return (
        os.path.join(root, file)
        for root, dirs, files in os.walk(basedir)
        for file in files if root == basedir and file.lower().endswith(endswith)
        )
def find_index_exceeding_value(lst, VAL):
    indices = [index for index, value in enumerate(lst) if value > VAL]
    return indices[0] if indices else None
basedir = "/Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/"


In [2]:
def read_and_prep_flux_var_data(flux_file, var_file, wl_low, wl_high):
    """ This method reads the flux and var cubes for a specific observation, and cleans them up before returning """
    hdr_f, flux = open_kcwi_cube(flux_file)
    # print(repr(hdr_f))
    _, var = open_kcwi_cube(var_file)
    wave = build_wave(hdr_f)
    low_index = find_index_exceeding_value(wave, wl_low)
    high_index = find_index_exceeding_value(wave, wl_high)
    print(f"low: {low_index}  high: {high_index}")
    print(f"wave[{low_index}]:{wave[low_index]}  wave[{high_index}]:{wave[high_index]}")
    wave = wave[low_index:high_index+1]
    print(f"wave[0]:{wave[0]}  wave[-1]:{wave[-1]}")
    # hdr_f["CD1_1"] = hdr_f["CDELT1"]
    # hdr_f["CD2_2"] = hdr_f["CDELT2"]
    # hdr_f["CD3_3"] = hdr_f["CDELT3"]

    q_n = np.isnan(flux)
    q_neg = np.where(flux < 0.0)
    flux[q_n] = 0.0
    #flux[q_neg] = 0.0

    q_v = np.isnan(var)
    q_vneg = np.where(var < 0.0)
    var[q_v] = 0.0
    #var[q_vneg] = 0.0

    wl = np.sum(flux, axis=0)
    wcs_f = WCS(hdr_f).celestial

    return flux, var, wave, wl, wcs_f, hdr_f


In [3]:
def label_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
        ax.tick_params(labelbottom=False, labelleft=False)

In [4]:
def create_images_layout(plt, count=6, shape=None, title="GridSpec Layout"):
    """returns an arround of {count} axes for images to be placed within"""
    fig = plt.figure(figsize=(12,10), layout="constrained")
    # fig.suptitle(title)
    axs_image = []
    gs = gridspec.GridSpec(10, count, figure=fig)
    for i in range(count):
        # print(f"index:{i}")
        axs_image.append(fig.add_subplot(gs[1:10, i]))
        label_axes(fig)

    return fig, axs_image

In [5]:
flux_files = list(sorted(find_fits(basedir,"_corrected_flux.fits")))
var_files = list(sorted(find_fits(basedir,"_corrected_var.fits")))
# zipped = zip(flux_files, var_files)
data = list(zip(flux_files, var_files))
print(f"We have {len(data)} files to process")
for ii, datum in enumerate(data):
    print(f"data[{ii}]: {datum[0]}:{datum[1]}")


We have 7 files to process
data[0]: /Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/KB.20180708.23055_icubes_corrected_flux.fits:/Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/KB.20180708.23055_icubes_corrected_var.fits
data[1]: /Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/KB.20180708.24349_icubes_corrected_flux.fits:/Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/KB.20180708.24349_icubes_corrected_var.fits
data[2]: /Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/KB.20190601.32100_icubes_corrected_flux.fits:/Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed/KB.20190601.32100_icubes_corrected_var.fits
data[3]: /Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research

In [6]:
def display_wl_image(fig, ax, wl_image, point, hdr_f):
    xc = [point[0]]
    yc = [point[1]]
    lw = 0.5
    box_size_arcsecond = 2

    cmap = 'gnuplot'

    # plt.close()
    im1 = ax.imshow(wl_image, origin='lower', interpolation='nearest', cmap=cmap, vmin=0, vmax=10)
    #plt.colorbar(im1)
    ax.plot(xc, yc, 'gs')
    ax.grid()
    box_size = int(box_size_arcsecond / (hdr_f["CD2_2"] * 3600))
    hb = box_size // 2
    ax.set_xlabel("RA")
    ax.set_ylabel("Decl.")
    for i in range(len(xc)):
        ax.plot([xc[i] - hb, xc[i] + hb, xc[i] + hb, xc[i] - hb, xc[i] - hb], 
                 [yc[i] - hb, yc[i] - hb, yc[i] + hb, yc[i] + hb, yc[i] - hb], 'w-', lw=lw+1)
    # fig.suptitle(k_fits_list[l][:24])
    fig.suptitle("just another plot")


In [7]:
def main_ish():
    basedir = "/Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed"
    flux_files = list(sorted(find_fits(basedir,"_corrected_flux.fits")))
    var_files = list(sorted(find_fits(basedir,"_corrected_var.fits")))
    zipped = zip(flux_files, var_files)
    print(repr(enumerate(zipped)))

    wl_low = 3500
    wl_high = 5500 
    
    spectrum_sum = None # this needs to have the same the specturm shape
    variance_sum = None # this needs to have the same the specturm shape
    axs = []
    data = list(zip(flux_files, var_files))
    count = len(data)
    fig, axs_image = create_images_layout(plt, count=count, shape=None, title="GridSpec Layout")
    for ii, datum in enumerate(data):
        # if ii > 0: break
        # print(f"flux_file: {datum[0]}")
        # print(f"var_file: {datum[1]}")
        flux, var, wave, wl_image, wcs_f, hdr_f = read_and_prep_flux_var_data(datum[0], datum[1], wl_low, wl_high)

        print("- "*20)
        # set up the layout
        # ax_image, ax_spec, ax_variance = create_image_flux_var_layout(plt, title="GridSpec Layout")
        # axs.append((ax_image, ax_spec, ax_variance))
        point=[17,54]
        # display the WL image
        display_wl_image(fig, axs_image[ii], wl_image, point, hdr_f)

        
        # display the point or points on the WL image
        # extract the weighted spectrum and variance
        # display the spectrum and variance
        # add spectrum / variance to 
    plt.show()
main_ish()


low: 271  high: 2271
wave[271]:3500.001162447039  wave[2271]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452


Set MJD-BEG to 58989.450463 from DATE-BEG.
Set MJD-END to 58989.464352 from DATE-END'. [astropy.wcs.wcs]
Set MJD-BEG to 58307.267941 from DATE-BEG.
Set MJD-END to 58307.281830 from DATE-END'. [astropy.wcs.wcs]


- - - - - - - - - - - - - - - - - - - - 
low: 272  high: 2272
wave[272]:3500.001162447039  wave[2272]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452
- - - - - - - - - - - - - - - - - - - - 
low: 272  high: 2272
wave[272]:3500.001162447039  wave[2272]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452


Set MJD-BEG to 58635.357647 from DATE-BEG.
Set MJD-END to 58635.371536 from DATE-END'. [astropy.wcs.wcs]
Set MJD-BEG to 58635.372209 from DATE-BEG.
Set MJD-END to 58635.386098 from DATE-END'. [astropy.wcs.wcs]


- - - - - - - - - - - - - - - - - - - - 
low: 272  high: 2272
wave[272]:3500.001162447039  wave[2272]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452
- - - - - - - - - - - - - - - - - - - - 
low: 271  high: 2271
wave[271]:3500.001162447039  wave[2271]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452


Set MJD-BEG to 58989.421057 from DATE-BEG.
Set MJD-END to 58989.434946 from DATE-END'. [astropy.wcs.wcs]
Set MJD-BEG to 58989.435739 from DATE-BEG.
Set MJD-END to 58989.449628 from DATE-END'. [astropy.wcs.wcs]


- - - - - - - - - - - - - - - - - - - - 
low: 271  high: 2271
wave[271]:3500.001162447039  wave[2271]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452
- - - - - - - - - - - - - - - - - - - - 
low: 271  high: 2271
wave[271]:3500.001162447039  wave[2271]:5500.527840926287
wave[0]:3500.001162447039  wave[-1]:5499.527573842452


Set MJD-BEG to 58989.450463 from DATE-BEG.
Set MJD-END to 58989.464352 from DATE-END'. [astropy.wcs.wcs]


- - - - - - - - - - - - - - - - - - - - 


: 